<a href="https://colab.research.google.com/github/LaibaKhan112/36-Weeks-Preparation-Challenge/blob/main/Function_Calling_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import userdata
from google import genai
from google.genai import types

# ---------- 1) API KEY + CLIENT ----------
API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=API_KEY)   # IMPORTANT: no "with ... as client" (it closes the client)

# ---------- 2) YOUR REAL PYTHON FUNCTIONS ----------
def enable_lights():
    print("LIGHTBOT : Lights enabled.")
    return "lights_enabled"

def set_light_color(rgb_hex: str):
    print(f"LIGHTBOT : Lights set to {rgb_hex}")
    return f"color_set_to_{rgb_hex}"

def stop_lights():
    print("LIGHTBOT : Lights off.")
    return "lights_off"

PY_TOOLS = {
    "enable_lights": enable_lights,
    "set_light_color": set_light_color,
    "stop_lights": stop_lights,
}

# ---------- 3) TOOL (FUNCTION) DECLARATIONS ----------
enable_lights_fn = {
    "name": "enable_lights",
    "description": "Turn on the lighting system",
    "parameters": {"type": "object", "properties": {}},
}

set_light_color_fn = {
    "name": "set_light_color",
    "description": "Set the light color using hex RGB value (example: #FFA500)",
    "parameters": {
        "type": "object",
        "properties": {"rgb_hex": {"type": "string"}},
        "required": ["rgb_hex"],
    },
}

stop_lights_fn = {
    "name": "stop_lights",
    "description": "Turn off the lighting system",
    "parameters": {"type": "object", "properties": {}},
}

tools = [
    types.Tool(
        function_declarations=[enable_lights_fn, set_light_color_fn, stop_lights_fn]
    )
]

config = types.GenerateContentConfig(
    tools=tools,
    system_instruction=(
        "You are a lighting system bot. "
        "If the user requests an action, you MUST call a tool. "
        "Never claim you did an action unless you called the tool."
    ),
)

# ---------- 4) CREATE A CHAT ----------
chat = client.chats.create(
    model="gemini-2.5-flash",
    config=config,
)





In [13]:
# ---------- 5) SIMPLE TOOL LOOP ----------
def ask(prompt: str):
    resp = chat.send_message(prompt)
    print(f"Response: {resp}")

    while True:
        part = resp.candidates[0].content.parts[0]

        # If no tool call, we are done
        if not getattr(part, "function_call", None):
            print("MODEL:", resp.text)
            return resp

        fc = part.function_call
        name = fc.name
        args = fc.args or {}

        print("TOOL CALL:", name, args)

        # Run python tool
        result = PY_TOOLS[name](**args) if args else PY_TOOLS[name]()
        if result is None:
            result = "ok"

        # ✅ Send tool result back as a Part (NOT Content)
        resp = chat.send_message(
            types.Part(
                function_response=types.FunctionResponse(
                    name=name,
                    response={"result": result},
                )
            )
        )

In [14]:
# ---------- 6) TEST ----------
ask("Turn the lights off?")

Response: sdk_http_response=HttpResponse(
  headers=<dict len=10>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        function_call=FunctionCall(
          args={},
          name='stop_lights'
        ),
        thought_signature=b'\n\xba\x01\x01r\xc8\xda|\xfe\xfc\x10VOo]\xb7\xd6\x83s\xe2\xbbgo\xd9\xc77V\xc10u\xca=\x9b\xa0tj0_G\x15\xfe\xb4\xf8E\xf04\xef3\x1b\xa4v1\xad\x88\x8d\x98\xc3\x03\xb9\xb0\x16\xb8].\xc2\x0c\xa5\x9aJK\xa9S\x0c9\xf6\xc5\xa6y\x99W\xf0\x9e\xbb7\xd0;c\xbf\xe1\x05{\xadrd\x86\xe5\x98...'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='m097aY6aKLOOjMcP7vjSgAs' usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=10,
  prompt_token_count=168,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=168
    ),
  ],
  thoughts_token_count=3

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="I've turned the lights off."
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='nE97aeWZCpix-sAPpNe5gQM',
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=8,
    prompt_token_count=195,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=195
      ),
    ],
    total_token_count=203
  )
)